# Imports necessary



> Here we can see all the libraries that were usefull to help us scrape the data from the news papers we considered interesting to our study.

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

import re

# MBL scraping

> First we thought it would be useful to scrape some icelandic news from the mbl.is website. This news are included in the categories politics_and_society, nature_and_travel, hot_news and culture_and_living.

In [ ]:
url = "https://icelandmonitor.mbl.is/news/"
r = urllib.request.urlopen(url).read()
soup = BeautifulSoup(r, "lxml")

In [ ]:
links = set()   # A set to save the links from each category

for i in soup.find_all("div", {'class': 'navbar-sections boc seen-0111'}):
  for j in i.findAll('a'):
    current = j.attrs['href']
    if current[:5] == '/news':
      links.add("https://icelandmonitor.mbl.is/" + current[1:])


  print(links)

{'https://icelandmonitor.mbl.is/news/category/eat_and_drink/', 'https://icelandmonitor.mbl.is/news/category/politics_and_society/', 'https://icelandmonitor.mbl.is/news/category/nature_and_travel/', 'https://icelandmonitor.mbl.is/news/latest/', 'https://icelandmonitor.mbl.is/news/category/culture_and_living/'}


In [ ]:
articles = set()
codes = {}
sublinks = set()


for link in links:
  sub_read = urllib.request.urlopen(link).read()
  sub_soup = BeautifulSoup(sub_read, "lxml")

  for href in sub_soup.find_all('div', {'class': 'fpc'}):
    hr = href.a.attrs["href"]

    if hr.split('/')[1] == "frettir":
       articles.add('https://www.mbl.is/frettir/' + '/'. join(hr.split('/')[2:]))
    else:
      articles.add(url[:-1] + '/' + '/'. join(hr.split('/')[2:]))

  category = link.split('/')[-2]

  # Below we are trying to access the load more button

  count = 0
  max_iter = 9

  while count <= max_iter:
    if category not in codes.keys():
      for code in sub_soup.find_all('div', {'class': 'news-list-wrapper'}):
        next_code = re.split("= |;", code.find_all('script')[1].text)[1]

      codes[category] = [next_code]

    else:
      l = 'https://icelandmonitor.mbl.is/news/gmnc/'+ category +'/?p='+ codes[category][-1]+'&id=newslist&count=4&group_id=-1'
      if category == 'latest':
        l = 'https://icelandmonitor.mbl.is/news/gmnl/?p=2644482&id=newslist&count=4&group_id=-1'
      sublinks.add(l)
      # print(l)
      subsub_read = urllib.request.urlopen(l).read()
      subsub_soup = BeautifulSoup(subsub_read, "lxml")
      for code in subsub_soup.find_all('script', {'type': 'text/javascript'}):
        next_code = re.split("= |;", code.text)[1]
        codes[category].append(next_code)
    count += 1

for link in sublinks:
  sub_read = urllib.request.urlopen(link).read()
  sub_soup = BeautifulSoup(sub_read, "lxml")

  for href in sub_soup.find_all('div', {'class': 'fpc'})[:10]:
    hr = href.a.attrs["href"]
    print(hr)
    print(hr.split('/')[1])
    articles.add(url[:-1] + '/' + '/'. join(hr.split('/')[2:]))







  # for code in codes:



# Dataframe itself
print(sublinks)
print(codes)

/news/eat_and_drink/2022/10/12/landed_the_second_prize_in_the_european_street_food/
news
/news/eat_and_drink/2022/10/02/carpe_diem_could_be_this_chef_s_motto/
news
/news/eat_and_drink/2022/09/18/i_m_in_love_with_your_country/
news
/news/eat_and_drink/2022/09/10/great_atmosphere_at_matey_food_festival/
news
/news/eat_and_drink/2022/09/08/want_to_put_the_westman_islands_on_the_culinary_map/
news
/news/eat_and_drink/2022/09/07/nothing_danish_there_with_all_due_respect/
news
/news/eat_and_drink/2022/09/06/the_chocolate_covered_liquorice_war/
news
/news/eat_and_drink/2022/09/04/north_has_been_fully_booked_since_it_opened/
news
/news/eat_and_drink/2022/08/05/vera_is_a_newest_foodhall_on_the_scene/
news
/news/eat_and_drink/2022/07/18/a_boutique_coffeehouse_in_grundarfjordur_with_coffe/
news
/news/eat_and_drink/2022/07/12/vogue_picks_the_top_bars_in_reykjavik/
news
/news/eat_and_drink/2022/06/29/heimkaup_starts_selling_wine_and_beer_with_same_day/
news
/news/eat_and_drink/2022/05/31/the_first_

In [ ]:
df = pd.DataFrame(columns=["Source", "Headline", "Datetime", "Body", 'Category'])

i = 0
for l in articles:
  current_read = urllib.request.urlopen(l).read()
  current_soup = BeautifulSoup(current_read, "lxml")

  headline = (current_soup.find_all("h1")[0].text).strip()
  dateline = current_soup.find_all("div", {"class": "dateline"})
  date = (((dateline[0].text).split("|")[2]).strip()).split(" ")[1:]
  hour = (((dateline[0].text).split("|")[3]).strip()).split(" ")[0:1]
  category = dateline[0].text.split('|')[0].strip()

  body = ""
  for s in current_soup.find_all("div", {"class": "maintext"}):
    body = body + s.text

  body = body.replace("\n", " ")
  body = ' '.join(body.split())
  current_dict = {"Source": "Iceland Monitor", "Headline": headline, "Datetime": date+hour, "Body": body, 'Category': category}
  df = df.append(current_dict, ignore_index=True)
  i+=1
  perc = i*100/len(articles)
  print(f'\n \tWe have already have {str(round(perc,2))}% of the data!! link: {l}\n')


<ipython-input-24-20d67951d720>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_dict, ignore_index=True)



 	We have already have 0.22% of the data!! link: https://icelandmonitor.mbl.is/news/nature_and_travel/2022/10/06/a_triceratop_dinosaur_coming_to_iceland/



<ipython-input-24-20d67951d720>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_dict, ignore_index=True)



 	We have already have 0.45% of the data!! link: https://icelandmonitor.mbl.is/news/eat_and_drink/2018/11/19/first_lady_to_accompany_icelandic_national_culinary/



<ipython-input-24-20d67951d720>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_dict, ignore_index=True)


...

<ipython-input-24-20d67951d720>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_dict, ignore_index=True)



 	We have already have 99.55% of the data!! link: https://icelandmonitor.mbl.is/news/eat_and_drink/2022/08/05/vera_is_a_newest_foodhall_on_the_scene/



<ipython-input-24-20d67951d720>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_dict, ignore_index=True)



 	We have already have 99.78% of the data!! link: https://icelandmonitor.mbl.is/news/culture_and_living/2021/09/18/harry_potter_stars_happy_with_iceland_visit/


 	We have already have 100.0% of the data!! link: https://icelandmonitor.mbl.is/news/culture_and_living/2022/08/16/reykjavik_celebrates_culture_night_2022_next_saturd/



<ipython-input-24-20d67951d720>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(current_dict, ignore_index=True)


## To Datetime format

> Here we transform our datetime column from a string fromat into a datetime fromat. This will be useful in the future.

In [ ]:
data = pd.read_csv('mbl_data.csv')
data.head()
data['Datetime'][0]

"['6', 'Oct', '2022', '23.31']"

In [ ]:
dfc['Datetime'].sort_values()

232   2017-06-04 09:00:00
83    2017-06-05 09:31:00
233   2017-06-07 20:00:00
103   2017-06-08 20:00:00
100   2017-06-09 20:12:00
              ...        
96    2023-03-28 01:47:00
401   2023-03-28 13:01:00
79    2023-03-28 16:00:00
303   2023-03-29 07:03:00
42    2023-03-29 12:00:00
Name: Datetime, Length: 445, dtype: datetime64[ns]

In [ ]:
data.isnull().sum()

Unnamed: 0    0
Source        0
Headline      0
Datetime      0
Body          1
Category      0
dtype: int64

In [ ]:
dfc = data

# dictionary to help identify the months using number from 1 to 12

month = {'Jan': 1,
         'Feb': 2,
         'Mar': 3,
         'Apr': 4,
         'May': 5,
         'Jun': 6,
         'Jul': 7,
         'Aug': 8,
         'Sep': 9,
         'Oct': 10,
         'Nov': 11,
         'Dec': 12}

for i in range(0, dfc.shape[0]):
  dfc.Datetime[i]  = re.sub(r"[,']", "", string = dfc.Datetime[i][1:-1]).split(' ')
  if len(dfc.Datetime[i]) != 4:
    dfc = dfc.drop([i])
  else:
    dfc.Datetime[i][1] = str(month[dfc.Datetime[i][1]])
    dfc.Datetime[i][0] =  '-'.join(dfc.Datetime[i][:3])
    dfc.Datetime[i] = str(dfc.Datetime[i][0]+ ' ' + dfc.Datetime[i][-1])

dfc['Datetime'] = pd.to_datetime(dfc['Datetime'].str.replace('.',':'), format="%d-%m-%Y %H:%M")

dfc.head()

<ipython-input-44-9de3500717e3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.Datetime[i]  = re.sub(r"[,']", "", string = dfc.Datetime[i][1:-1]).split(' ')
<ipython-input-44-9de3500717e3>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.Datetime[i] = str(dfc.Datetime[i][0]+ ' ' + dfc.Datetime[i][-1])
<ipython-input-44-9de3500717e3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.Datetime[i]  = re.sub(r"[,']", "", str

,Unnamed: 0,Source,Headline,Datetime,Body,Category
0,0,Iceland Monitor,A Triceratop Dinosaur coming to Iceland?,2022-10-06 23:31:00,A part of a dinosaur skeleton could be on its ...,Nature and Travel
1,1,Iceland Monitor,First Lady to accompany Icelandic National Cul...,2018-11-19 12:08:00,The Icelandic National Culinary team will be c...,Eat and Drink
2,2,Iceland Monitor,Successful Landing Exercise in Hvalfjörður Fjo...,2022-04-12 10:27:00,Everything went according to plan during the l...,Politics and Society
3,3,Iceland Monitor,Heimkaup starts selling wine and beer with sam...,2022-06-29 11:41:00,Today is the first day you can buy wine and be...,Eat and Drink
4,4,Iceland Monitor,Sheep Eat Arctic Tern Eggs and Chicks,2022-01-21 12:35:00,A newly published article in the nature journa...,Nature and Travel


In [ ]:
dfc.to_csv('mbl_data.csv')

# Central Bank Iceland

> Given that that we want to predict the ISK/EUR and ISK/USD value it may be of interest to start  by scraping the news from the icelandic central bank.

In [ ]:
url = "https://www.cb.is/publications/news/news-by-year/?news_ArchiveDate=2022-01-01&news_date=2022-01-01"
r = urllib.request.urlopen(url).read()
soup = BeautifulSoup(r, "lxml")

In [ ]:
links = set()
for i in soup.find_all("a"):
  current = i.attrs["href"]
  if (current.count("/") > 6):
    links.add("https://www.cb.is" + current)

In [ ]:
df = pd.DataFrame()
for i, l in enumerate(links):
  r = urllib.request.urlopen(l).read()
  soup = BeautifulSoup(r, "lxml")
  title = soup.find_all("title")[0].text
  for j in soup.find_all("div", {"class": "media"}):
    date = j.attrs['data-last-modified']
  for p in soup.find_all("div", {"class": "media"}):
    alltext = p.text
    body = alltext.split(title)[1]
  df = pd.concat([df, pd.DataFrame({"headline": title, "date": date, "body": body}, index=[i])])

In [ ]:
df.drop_duplicates(subset='headline', keep='first', inplace=True)
df.to_csv("cb_data.csv")

# Scraping Central banks using the guardian


> We decided to consider central bank news as highly relevant news. First we decided to scrape the central bank own website but we got the response status code 403 in the federal reserve (USA central bank) website and the european central bank despite not showing the 403 status code it seemed that we couldn't access the news using beautiful soup. Hence we had to go look for some second hand news about these two central banks. For this task we found "The Guardian" webiste to be very useful and that is what we decided to scrape.

In [ ]:
def scrape_relevant_data(max_pages, region):
  """
  This function scrapes possibly relevant and informative central bank news to our future models. The website used for this was the newspaper "The Guardian".
  Input:
    - max_pages: number of pages to scrape. It should be bigger than 1.
    -region: Central bank in which the user is interested in scraping. It can assume the values "EU" or "USA"
  Output:
    - News articels organized in a dataset with the collumns date, headline, body and region.
  """


  if region == 'EU':
    central_bank = 'european-central-bank'
  elif region == 'USA':
    central_bank = 'federal-reserve'

  headline = []
  body = []
  date = []


  # Scraping the link for each article
  links = []
  b = 0
  for page_number in range(1, max_pages):
    url = f'https://www.theguardian.com/business/{central_bank}?page={page_number}'
    r = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")

    for element in soup.find_all('div', {'class': 'fc-item__container'}):
      links.append(element.a.attrs['href'])
      # print(element)
      date.append(element.find('time', {'class': 'fc-item__timestamp'}).attrs['datetime'])

    for head in soup.find_all('span', {'class': 'js-headline-text'}):

      headline.append(head.text)

  for article in links:

    r = urllib.request.urlopen(article).read()
    # print(article, head, time)
    mini_soup = BeautifulSoup(r, "lxml")
    if mini_soup.find('div', {'class': 'article-body-commercial-selector'}):
      for text in mini_soup.find_all('div', {'class': 'article-body-commercial-selector'}):
          news_art = text.text
    elif mini_soup.find_all('article', {'class': 'block'}):
      for text in mini_soup.find_all('article', {'class': 'block'}):
        news_art = text.text
    else:
      news_art = ''


    body.append(news_art)

  dictionary = {'date': date, 'headline':list(headline), 'body':body, 'origin': [region]*len(headline)}

  return pd.DataFrame(dictionary)

  # for link in links



In [ ]:
df_eu = scrape_relevant_data(max_pages = 50, region = 'EU') # scraping first 50 pages
df_eu = df_eu.drop_duplicates()
df_eu

https://www.theguardian.com/business/2023/mar/31/eurozone-inflation-falls-sharply-as-energy-prices-drop
https://www.theguardian.com/business/2023/mar/22/interest-rate-rise-us-federal-reserve-banking-turmoil
https://www.theguardian.com/business/2023/mar/17/credit-suisse-shares-continue-to-fall-despite-efforts-to-calm-nerves
https://www.theguardian.com/business/2023/mar/16/ecb-raises-interest-rates-despite-banking-fears-credit-suisse
https://www.theguardian.com/business/2023/mar/16/ecb-interest-rate-rise-credit-suisse-european-central-bank
https://www.theguardian.com/business/2023/mar/02/ecb-looking-out-for-price-gouging-greedflation-price-rises-eurozone-inflation-profit-margins
https://www.theguardian.com/business/live/2023/jan/20/davos-day-4-global-economic-outlook-russia-economy-ai-business-live
https://www.theguardian.com/business/2023/jan/20/climate-economic-stability-central-bankers
https://www.theguardian.com/business/2023/jan/10/world-bank-walking-tightrope-as-it-mulls-increased-

In [ ]:
df_usa = scrape_relevant_data(max_pages = 50, region = 'USA')
df_usa = df_usa.drop_duplicates()
df_usa

https://www.theguardian.com/business/2023/apr/07/us-jobs-report-march-2023
https://www.theguardian.com/business/2023/mar/22/interest-rate-rise-us-federal-reserve-banking-turmoil
https://www.theguardian.com/business/2023/mar/22/decision-on-interest-rate-rise-on-knife-edge-amid-banking-crisis-fears
https://www.theguardian.com/us-news/2023/mar/21/federal-reserve-new-york-union-sodexo-food-service
https://www.theguardian.com/commentisfree/2023/mar/21/federal-reserve-interest-rate-hikes-central-banks
https://www.theguardian.com/business/2023/mar/19/elizabeth-warren-us-bank-collapse-fed-chair-jerome-powell
https://www.theguardian.com/business/2023/mar/19/silicon-valley-banks-collapse-will-not-be-a-one-off-a-banking-crisis-was-long-overdue
https://www.theguardian.com/commentisfree/2023/mar/17/silicon-valley-bank-bailout-dodd-frank-regulation-opinion
https://www.theguardian.com/business/2023/mar/16/cash-strapped-banks-have-borrowed-300bn-from-the-fed-this-past-week
https://www.theguardian.com/

> Below we concat the USA and EU data in one dataframe.

In [ ]:
df_central_banks = pd.concat([df_eu, df_usa])
df_central_banks.to_csv('Central-bank-news.csv')
df_central_banks = df_central_banks.drop_duplicates()

In [ ]:
df_central_banks = df_central_banks.reset_index()
df_central_banks['is_relevant'] = [1] * df_central_banks.shape[0]

In [ ]:
df_central_banks = df_central_banks.drop(['level_0',
                                          'index'], axis = 1)
df_central_banks

,date,headline,body,origin,is_relevant
0,2023-03-31T11:58:03+0000,Eurozone inflation falls sharply as energy pri...,The eurozone’s annual inflation rate fell shar...,EU,1
1,2023-03-22T19:20:33+0000,US Federal Reserve raises interest rates a qua...,Facing the worst banking crisis since 2008 and...,EU,1
2,2023-03-17T17:55:05+0000,Credit Suisse shares continue to fall despite ...,Credit Suisse shares came under renewed pressu...,EU,1
3,2023-03-16T17:30:12+0000,ECB raises eurozone interest rate despite bank...,The European Central Bank has raised interest ...,EU,1
4,2023-03-16T10:47:35+0000,ECB faces dilemma over interest rate rise amid...,The European Central Bank is facing a dilemma ...,EU,1
...,...,...,...,...,...
1955,2012-06-20T15:04:00+0000,Market jitters ahead of Federal Reserve announ...,US stock markets wobbled Wednesday morning as ...,USA,1
1956,2012-06-18T17:24:34+0000,Fed could boost US economy with Operation Twis...,The Federal Reserve could step in with a new r...,USA,1
1957,2012-06-18T16:22:17+0000,The risk Jamie Dimon poses to the Federal Rese...,The Federal Reserve System was created in 1913...,USA,1
1958,2012-06-18T13:30:01+0000,What Ben Bernanke and the Federal Reserve can ...,The Federal Reserve Board's open market commit...,USA,1


# Scraping non-relevant news

> To feed into the model 1 (predicting the relevancy of an article) we alo need some non-relevant news. For this task it was decided to scrape news from the main categories on "The Guardian" that were not relevant at all. These categories are "sport", "culture" and "life and style" and they include both news from the United States and the Eropean Union.

In [ ]:
def non_relevant_news(max_pages, category):
  """
  It scrapes some non relevant data to train the first model used.

  Input:
    max pages = value bigger than one that represents the number of pages to scrape in each category
    categories_to_scrape = which categories should we scrape. It has to be a list that can assume the following values (['sport', 'cultural', 'lifeandstyle']).

  Output: Scraped Dataframe for the specified category.

  """
  date = []
  headline = []
  body = []

  for page_number in range(1, max_pages):
    url = f'https://www.theguardian.com/{category}/all?page={page_number}'
    r = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")

    links = []

    for element in soup.find_all('div', {'class': 'fc-item__container'}):
      if not element.find('div',{'class': 'fc-item__kicker'}) or element.find('div',{'class': 'fc-item__kicker'}) != 'Live':
        links.append(element.a.attrs['href'])
      date.append(element.find('time', {'class': 'fc-item__timestamp'}).attrs['datetime'])


    for head in soup.find_all('span', {'class': 'js-headline-text'}):
      headline.append(head.text)

    for article in links:
      r = urllib.request.urlopen(article).read()
      mini_soup = BeautifulSoup(r, "lxml")

      # A small condition to deal with different types of articles and to prevent
      # possible empty articles to mess up the order of the dataset

      if mini_soup.find('div', {'class': 'article-body-commercial-selector'}):
        for text in mini_soup.find_all('div', {'class': 'article-body-commercial-selector'}):
            news_art = text.text
      elif mini_soup.find_all('article', {'class': 'block'}):
        for text in mini_soup.find_all('article', {'class': 'block'}):
          news_art = text.text
      elif mini_soup.find('div', {'class': 'content__standfirst'}):
        for text in mini_soup.find_all('div', {'class': 'content__standfirst'}):
          news_art = text.text
      else:
        news_art = ''   # empty articles

      body.append(news_art)
  dictionary = {'date': date, 'headline':list(headline), 'body':list(body), 'origin': [category]*len(headline)}

  return pd.DataFrame(dictionary)


In [ ]:
df_sport = non_relevant_news(max_pages = 27, category = 'sport')

180
520 520 520


In [ ]:
df_culture = non_relevant_news(max_pages = 27, category = 'culture')

180
520 520 520


In [ ]:
df_lifestyle = non_relevant_news(max_pages = 27, category = 'lifeandstyle')

180
520 520 520


In [ ]:
df_non_relevant = pd.concat([
    df_sport,
    df_culture,
    df_lifestyle
])
df_non_relevant = df_non_relevant.reset_index()
df_non_relevant['is_relevant'] = [0] * df_non_relevant.shape[0]
df_non_relevant = df_non_relevant.drop('index', axis = 1)

In [ ]:
df_non_relevant.head()

,date,headline,body,origin,is_relevant
0,2023-04-07T22:06:42+0000,Brooks Koepka forges Masters lead while Rory M...,Seventeen shots separated Brooks Koepka and Ro...,sport,0
1,2023-04-07T21:51:14+0000,The Masters 2023: second round – as it happened,3h ago19.40 BSTRory McIlroy’s race is surely r...,sport,0
2,2023-04-07T21:30:04+0000,Pep Guardiola reveals shift in formation as re...,Pep Guardiola says he has faith in Kyle Walker...,sport,0
3,2023-04-07T21:30:03+0000,"‘Wow, where are we?’: Jürgen Klopp reflects on...",Jürgen Klopp feels Liverpool have been sucked ...,sport,0
4,2023-04-07T21:30:03+0000,Manchester United’s trophy push boosted by Ch...,Erik ten Hag has urged Manchester United to pu...,sport,0
...,...,...,...,...,...
1555,2023-03-06T08:00:21+0000,A new start after 60: I retired – and began to...,Kathi Muhammad turned 60 some years ago. She d...,lifeandstyle,0
1556,2023-03-06T07:00:20+0000,Most fortysomethings feel much younger than th...,"Last Saturday, I went birdwatching, poked the ...",lifeandstyle,0
1557,2023-03-06T07:00:20+0000,‘No problemo’: what Gen Z are really saying wi...,Despite all manner of global catastrophes faci...,lifeandstyle,0
1558,2023-03-06T06:30:19+0000,"Sunak, Starmer and supermarket falafel: Edith ...",1 Sunak v Starmer/Supermarket falafel: Didn't ...,lifeandstyle,0


In [ ]:
df_relevancy = pd.concat([df_non_relevant, df_central_banks])
df_relevancy = df_relevancy.reset_index().drop('index', axis = 1)
df_relevancy.to_csv('relevancy_dataset.csv')
df_relevancy

,date,headline,body,origin,is_relevant
0,2023-04-07T22:06:42+0000,Brooks Koepka forges Masters lead while Rory M...,Seventeen shots separated Brooks Koepka and Ro...,sport,0
1,2023-04-07T21:51:14+0000,The Masters 2023: second round – as it happened,3h ago19.40 BSTRory McIlroy’s race is surely r...,sport,0
2,2023-04-07T21:30:04+0000,Pep Guardiola reveals shift in formation as re...,Pep Guardiola says he has faith in Kyle Walker...,sport,0
3,2023-04-07T21:30:03+0000,"‘Wow, where are we?’: Jürgen Klopp reflects on...",Jürgen Klopp feels Liverpool have been sucked ...,sport,0
4,2023-04-07T21:30:03+0000,Manchester United’s trophy push boosted by Ch...,Erik ten Hag has urged Manchester United to pu...,sport,0
...,...,...,...,...,...
3515,2012-06-20T15:04:00+0000,Market jitters ahead of Federal Reserve announ...,US stock markets wobbled Wednesday morning as ...,USA,1
3516,2012-06-18T17:24:34+0000,Fed could boost US economy with Operation Twis...,The Federal Reserve could step in with a new r...,USA,1
3517,2012-06-18T16:22:17+0000,The risk Jamie Dimon poses to the Federal Rese...,The Federal Reserve System was created in 1913...,USA,1
3518,2012-06-18T13:30:01+0000,What Ben Bernanke and the Federal Reserve can ...,The Federal Reserve Board's open market commit...,USA,1
